## Tutorial
The basic idea is :


```
from ultralytics import YOLO

model = YOLO("yolov11n.pt") # We are going off the pretrained model, not starting from scratch

model.train(data="dataset_folder/data.yaml", epochs=1000, batch=-1, device=0, cache=True, classes=None, patience=50)
# A YOLO dataset is a folder with a data.yaml file giving the params to the model
# epochs = 1000 should ensure the model keeps on training until it judges itself to have attained its best possible performance
# batch = -1 allows the model engine to autodetermine the best batching size (parallelizing inference). Use 0.55 (55%) if VRAM runs out (lower if still runs out)
# device=0 ensures that the model runs on the first GPU available (if more than one : device=[0,1,...]
# device=cpu would allow training on cpu if no GPU available
# by default this script runs on first available gpu, defaults to cpu if none
#cache=True means the images are cached instead of loaded from disk on each pass (make sure enough RAM available)'
# Classes = None, means training on all classes, classes=list[int] will only train on the classes in the list

```

After this process, the trained model is at `runs/detect/train/weights/best.pt`




# Train YOLO Object Detection on a Custom Dataset

---


## Inputs and parameters

In [1]:
model_base = "yolo11s.pt"
export_formats = ["onnx", "openvino", "torchscript"]
export_zip_name = "trained_model_s_nounknown.zip"
training_params = {
    "epochs" : 1000,
    "batch" : -1,
    "cache" : True,
    "imgsz" : 640,
    "classes" : [0,1,2,3,4,5,6,7,8,9,11] #everything but unknown
}

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
dataset_zip = "/content/drive/MyDrive/Colab Notebooks/datasets/dataset.zip"

**ALWAYS GO CHECK THE PATH IN THE BEFORE LAST CELL, BY DEFAULT THIS WILL SAVE THE MODELS IN RUNS>DETECT>TRAIN2 !**

## Device setup

make sure we have access to GPU

In [4]:
from tensorflow.test import gpu_device_name

DEVICE = gpu_device_name()
if len(DEVICE) > 0:
  DEVICE = DEVICE[12:]
else:
  DEVICE = "cpu"

In [5]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLO


In [6]:
!pip install ultralytics

Check that YOLO runs and all required librairies are present

In [7]:
from IPython.display import display, Image, clear_output
clear_output()
!yolo check

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 33.3/112.6 GB disk)

OS                  Linux-6.1.85+-x86_64-with-glibc2.35
Environment         Colab
Python              3.11.11
Install             pip
RAM                 52.96 GB
Disk                33.3/112.6 GB
CPU                 Intel Xeon 2.20GHz
CPU count           12
GPU                 NVIDIA L4, 22693MiB
GPU count           1
CUDA                12.4

numpy               ✅ 1.26.4<=2.1.1,>=1.23.0
matplotlib          ✅ 3.10.0>=3.3.0
opencv-python       ✅ 4.11.0.86>=4.6.0
pillow              ✅ 11.1.0>=7.1.2
pyyaml              ✅ 6.0.2>=5.3.1
requests            ✅ 2.32.3>=2.23.0
scipy               ✅ 1.13.1>=1.4.1
torch               ✅ 2.5.1+cu124>=1.8.0
torch               ✅ 2.5.1+cu124!=2.4.0,>=1.8.0; sys_platform == "win32"
torchvision         ✅ 0.20.1+cu124>=0.9.0
tqdm                ✅ 4.67.1>=4.64.0
psutil              ✅ 5.9.5
py-cpuinfo          ✅ 9

In [8]:
#!pip install roboflow

#from roboflow import Roboflow
#rf = Roboflow(api_key="obcyD7henw5LjBBuINZP")
#project = rf.workspace("projet-industrie").project("traffic-counting-singapore")
#version = project.version(3)
#dataset = version.download("yolov11")

In [9]:
from ultralytics import YOLO
import wandb
wandb.init(mode="disabled")

## Training

### Open dataset

In [10]:
from ultralytics.utils.downloads import unzip_file

dataset_dir = unzip_file(dataset_zip, path="/content/datasets")
print(f'{dataset_dir=}')
files = os.listdir(dataset_dir)
print(f'{files=}')
dataset_yaml = next((f for f in files if f.endswith('.yaml')), None)
dataset_yaml = os.path.join(dataset_dir, dataset_yaml)
print(f'{dataset_yaml=}')

WARNING ⚠️ Skipping /content/drive/MyDrive/Colab Notebooks/datasets/dataset.zip unzip as destination directory /content/datasets/dataset is not empty.
dataset_dir=PosixPath('/content/datasets/dataset')
files=['val', 'test', 'train', 'data.yaml']
dataset_yaml='/content/datasets/dataset/data.yaml'


### Run training

In [11]:
model = YOLO(model_base)

results = model.train(data=dataset_yaml, epochs=training_params['epochs'], batch=training_params['batch'], device=DEVICE, cache=training_params['cache'], imgsz=training_params['imgsz'], classes=training_params['classes'])
for format in export_formats:
  model.export(format=format)

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/datasets/dataset/data.yaml, epochs=1000, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=True, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11], retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_

train: Scanning /content/datasets/dataset/train/labels... 533 images, 0 backgrounds, 0 corrupt: 100%|██████████| 533/533 [00:00<00:00, 1077.20it/s]


train: New cache created: /content/datasets/dataset/train/labels.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.3GB RAM): 100%|██████████| 533/533 [00:00<00:00, 626.31it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA L4) 22.16G total, 0.14G reserved, 0.11G allocated, 21.91G free


...

                   all        133       5216      0.914      0.776      0.841      0.656
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 745, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

845 epochs completed in 2.140 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 19.3MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 19.3MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11s summary (fused): 238 layers, 9,417,444 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.06s/it]


                   all        133       5216        0.9      0.773      0.836      0.658
               Bicycle         61         88      0.853      0.761      0.809      0.649
                   Bus         47        120      0.964      0.942      0.963      0.898
                   Car        110       2571      0.924      0.792       0.87      0.687
         Flatbed truck         64         66      0.908      0.848      0.912      0.658
               Minibus         89        318      0.922      0.796      0.854      0.705
            Motorcycle        130        453      0.852      0.695      0.768      0.558
            Pedestrian        106       1036      0.791      0.329      0.449      0.255
              Rickshaw          7          7      0.853      0.833      0.843      0.594
               Scooter         45        135      0.927      0.658      0.806      0.562
                 Truck         66        172      0.976      0.965      0.985        0.9
                   Va

<frozen importlib.util>:262: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.


OpenVINO: export success ✅ 9.8s, saved as 'runs/detect/train2/weights/best_openvino_model/' (36.3 MB)

Export complete (10.3s)
Results saved to /content/runs/detect/train2/weights
Predict:         yolo predict task=detect model=runs/detect/train2/weights/best_openvino_model imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train2/weights/best_openvino_model imgsz=640 data=/content/datasets/dataset/data.yaml  
Visualize:       https://netron.app
Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
YOLO11s summary (fused): 238 layers, 9,417,444 parameters, 0 gradients, 21.3 GFLOPs

PyTorch: starting from 'runs/detect/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 16, 8400) (18.4 MB)

TorchScript: starting export with torch 2.5.1+cu124...
TorchScript: export success ✅ 2.5s, saved as 'runs/detect/train2/weights/best.torchscript' (36.5 MB)

Export complete (3.1s)
Results saved to /content/runs/detect/train2/

In [12]:
# Validate the model
metrics = model.val(save_json=True)  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

#this will create a new folder with the results of this val (usually train12, train22, train32 etc..), it can be discarded as they are also included in the original training folder

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11s summary (fused): 238 layers, 9,417,444 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /content/datasets/dataset/val/labels.cache... 133 images, 0 backgrounds, 0 corrupt: 100%|██████████| 133/133 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.1GB RAM): 100%|██████████| 133/133 [00:00<00:00, 768.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.32s/it]


                   all        133       5216        0.9      0.775      0.837      0.661
               Bicycle         61         88      0.853      0.761      0.809      0.649
                   Bus         47        120      0.972       0.95      0.969        0.9
                   Car        110       2571      0.923      0.793       0.87      0.691
         Flatbed truck         64         66      0.907      0.848      0.912      0.659
               Minibus         89        318      0.921      0.796      0.854      0.708
            Motorcycle        130        453      0.845      0.695      0.765      0.562
            Pedestrian        106       1036      0.795      0.329      0.449      0.258
              Rickshaw          7          7      0.853      0.835      0.843      0.594
               Scooter         45        135      0.925      0.667      0.814      0.571
                 Truck         66        172      0.973      0.965      0.985      0.902
                   Va

array([    0.64944,     0.90004,     0.69071,     0.65933,     0.70811,     0.56197,     0.25763,     0.59431,     0.57096,     0.90201,     0.66075,     0.77374])

# Download results

In [13]:
from ultralytics.utils.downloads import zip_directory
zipped_res = zip_directory('/content/runs/detect/train2')

Deleting .DS_Store files: []
Deleting __MACOSX files: []


Zipping /content/runs/detect/train2 to /content/runs/detect/train2.zip...: 100%|██████████| 26/26 [00:08<00:00,  3.11file/s]


In [14]:
from shutil import copy2
os.makedirs('/content/drive/MyDrive/Colab Notebooks/models', exist_ok=True)
copy2(zipped_res, os.path.join('/content/drive/MyDrive/Colab Notebooks/models', export_zip_name))

'/content/drive/MyDrive/Colab Notebooks/models/trained_model_s_nounknown.zip'

In [ ]:
from google.colab import runtime
runtime.unassign()